__Author: Sayemul Islam (ECE dept., Temple University)__

In [1]:
# VISA Resource Name: USB0::0xF4ED::0xEE3A::447K15118::INSTR

import visa #pip install pyvisa
from datetime import datetime
import time
import openpyxl # Excel processor library
import os # To open any files in file browser
import sys # Interfacing computer HID system
import readchar # pip install readchar
import numpy as np
from decimal import Decimal
import itertools
import winsound # To access windows default audio library 
import matplotlib.pyplot as plt
from matplotlib.mlab import find
%matplotlib inline

excelfile = 'CapturedData.xlsx'

# Connected VISA device should be displayed here:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL1::INSTR', 'ASRL3::INSTR')

In [2]:
def Excel_clear():
    i = 0
    # Excel Setup
    wb = openpyxl.load_workbook(excelfile)
    sheet = wb.active

    ######## Headings ##########
    sheet['A1'] = 'Last Updated:'
    sheet['A2'] = 'Iteration' # R1
    sheet['B2'] = 'DC Voltage' # R2
    sheet['C2'] = 'Time Stamp' # R3
    ###########################

    for row in sheet['A3:K10000']:
        for cell in row:
            cell.value = None 
    wb.save(excelfile)

In [1]:
dmm = rm.open_resource('USB0::0x0957::0x0618::TW46510051::0::INSTR') # Station 01's address

print(dmm.query("*IDN?"))

In [118]:
def Measure():
    global DC_V
    DC_V = dmm.query_ascii_values(':MEAS:VOLT:DC?')
    DC_V = DC_V[0] 
    return DC_V    

In [119]:
time_in_min = []
DC_Volt = []
i=0
rate = 10 # data per min (must be less than 60)
duration = 5 # minutes

iterate = rate*duration

Excel_clear() # Clear previous measurements

for i in range(0, iterate, 1): # Iterating i here <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    start = time.perf_counter()
    Measure()
    time_in_min.append(i)
    DC_Volt.append(DC_V)
    #write to excel
    sheet.cell(row=3+i, column=1).value=i+1
    sheet.cell(row=3+i, column=2).value=DC_V
    sheet.cell(row=3+i, column=3).value=datetime.now()
    sheet['C1'] = datetime.now() # Last Updated Time Stamp
    wb.save(excelfile)
    time.sleep((60/rate))
    stop = time.perf_counter()
    runtime = stop-start
    print("Scan:", i+1, ', DC Voltage: ' + str(DC_V)+' V, time:', runtime)

winsound.PlaySound("SystemHand", winsound.SND_ALIAS)
os.startfile(excelfile)

time_min = [i + 1 for i in time_in_min]
time_frac = [i / rate for i in time_min]

plt.figure(figsize=(8,6))
line1 = plt.plot(time_frac,DC_Volt,'o-r',linewidth='3',label='Voltage',ms=8)
plt.legend()
plt.xlabel('Time (Minutes)',fontsize=14)
plt.ylabel('Voltage (V)',fontsize=14)
plt.grid()
plt.tick_params(labelsize = 14)
plt.savefig('Plot_1.png', dpi=600,bbox_inches='tight')
plt.show()

dmm.close()
rm.close()

Scan: 1 , DC Voltage: 1 V, time: 10.208947099999932
Scan: 2 , DC Voltage: 2 V, time: 10.15103789999921
Scan: 3 , DC Voltage: 5 V, time: 10.145998399999371
Scan: 4 , DC Voltage: 10 V, time: 10.1938865000011
Scan: 5 , DC Voltage: 17 V, time: 10.14476730000024
Scan: 6 , DC Voltage: 26 V, time: 10.144118600001093
Scan: 7 , DC Voltage: 37 V, time: 10.194638699998904
Scan: 8 , DC Voltage: 50 V, time: 10.159754899999825
Scan: 9 , DC Voltage: 65 V, time: 10.192581299999802
Scan: 10 , DC Voltage: 82 V, time: 10.19559659999868
Scan: 11 , DC Voltage: 101 V, time: 10.178148000000874
Scan: 12 , DC Voltage: 122 V, time: 10.184336199999962
